# Reconhecimento de entidades nomeadas

Uma grande empresa de advocacia está buscando soluções tecnológicas para otimizar o processamento de documentos jurídicos e melhorar a gestão dos processos em andamento.

Atualmente, o escritório lida com um volume significativo de contratos, petições, jurisprudências e outros documentos legais, mas não há uma forma eficiente de extrair automaticamente informações críticas desses textos, como nomes de partes envolvidas, datas relevantes e dispositivos legais citados.

Nosso papel nesse projeto é desenvolver um modelo de IA especializado em Reconhecimento de Entidades Nomeadas (NER), capaz de identificar e classificar automaticamente essas informações em textos jurídicos. Isso permitirá que os advogados e equipes jurídicas tenham acesso mais rápido a dados importantes, otimizando o tempo de análise e reduzindo o risco de erro humano, além de facilitar a gestão dos processos de forma mais eficiente.

## Leitura dos textos

Vamos utilizar dados de arquivos txt que estão compactados em um arquivo zip. Para abrir esses arquivos, vamos usar a biblioteca `zipfile`.

In [1]:
import zipfile

In [2]:
import os

zip_path = '/content/sample_data/texts.zip'  # Definindo zip_path conforme especificado

print(os.getcwd())  # Imprimindo o diretório de trabalho atual

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip:
        print(zip.namelist())
else:
    print(f"Arquivo '{zip_path}' não encontrado.")

/content
['AC1TCU.txt', 'AC1TJAC.txt', 'AC1TJMG.txt', 'AC2.txt', 'ACORDAOTCU25052016.txt', 'ADI2TJDFT.txt', 'adi3767.txt', 'Ag10000170733596001.txt', 'Ag10105170208398001.txt', 'AgAIRR11889820145030011.txt', 'AgCr10582160008758001.txt', 'AgRgSTJ1.txt', 'AgRgSTJ2.txt', 'AgRgTSE1.txt', 'AgRgTSE3.txt', 'AIAgRAgI6193ARAGUARIMG.txt', 'AIRR15708820115050222.txt', 'AIRR3999520145020086.txt', 'airr801422012.txt', 'AP00000794920137060006.txt', 'AP00001415620157010201.txt', 'AP00001441420167030203.txt', 'AP771420167080008PA.txt', 'CP32320177080008PA.txt', 'DespSEPLAGDF.txt', 'ED1STM.txt', 'ED1TJAC.txt', 'EDAgRgTSE2.txt', 'EDEDARR208420135040232.txt', 'EDRR1TST.txt', 'EEDRR9715120105020002.txt', 'ERR731004520105130003.txt', 'HC110260SP.txt', 'HC151914AgRES.txt', 'HC340624SP.txt', 'HC418951PR.txt', 'HC70000845920187000000.txt', 'lei11340.txt', 'Lei11788.txt', 'LoaDF2018.txt', 'Pet128TSE5.txt', 'Port77DF.txt', 'Rcl3495STJ.txt', 'REE5908TSE4.txt', 'REsp1583083RS.txt', 'RR2574407120025020372.txt', 'R

In [3]:
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip:
        print(*zip.namelist(), sep='\n')
else:
    print(f"Arquivo '{zip_path}' não encontrado.")

AC1TCU.txt
AC1TJAC.txt
AC1TJMG.txt
AC2.txt
ACORDAOTCU25052016.txt
ADI2TJDFT.txt
adi3767.txt
Ag10000170733596001.txt
Ag10105170208398001.txt
AgAIRR11889820145030011.txt
AgCr10582160008758001.txt
AgRgSTJ1.txt
AgRgSTJ2.txt
AgRgTSE1.txt
AgRgTSE3.txt
AIAgRAgI6193ARAGUARIMG.txt
AIRR15708820115050222.txt
AIRR3999520145020086.txt
airr801422012.txt
AP00000794920137060006.txt
AP00001415620157010201.txt
AP00001441420167030203.txt
AP771420167080008PA.txt
CP32320177080008PA.txt
DespSEPLAGDF.txt
ED1STM.txt
ED1TJAC.txt
EDAgRgTSE2.txt
EDEDARR208420135040232.txt
EDRR1TST.txt
EEDRR9715120105020002.txt
ERR731004520105130003.txt
HC110260SP.txt
HC151914AgRES.txt
HC340624SP.txt
HC418951PR.txt
HC70000845920187000000.txt
lei11340.txt
Lei11788.txt
LoaDF2018.txt
Pet128TSE5.txt
Port77DF.txt
Rcl3495STJ.txt
REE5908TSE4.txt
REsp1583083RS.txt
RR2574407120025020372.txt
RR474820145230056.txt
RR942006420095040028.txt
TCU4687.txt
TSTRR16037920105200001.txt


In [4]:
with zipfile.ZipFile(zip_path, 'r') as zip:
    with zip.open('ADI2TJDFT.txt') as arquivo:
      texto = arquivo.read().decode('utf-8')

In [5]:
print(texto)

Órgão	:	Conselho Especial
Classe	:	ADI  Ação Direta de Inconstitucionalidade
N. Processo	:	2010002019357-4
Requerente(s)	:	PROCURADORA-GERAL DE JUSTIÇA DO DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s)	:	PRESIDENTE DA CÂMARA LEGISLATIVA DO DISTRITO FEDERAL E OUTRO(S)
Relator 	:	Desembargador LÉCIO RESENDE
	EMENTA	
AÇÃO DIRETA DE INCONSTITUCIONALIDADE. LEIS DISTRITAIS N.º 747/1994 E 2018/1998. LEI COMPLEMENTAR DISTRITAL N.º 380/2001. INCONSTITUCIONALIDADE FORMAL. LEI ORGÂNICA DO DISTRITO FEDERAL. OCUPAÇÃO DE ÁREA PÚBLICA. COMPETÊNCIA PRIVATIVA DO GOVERNADOR DO DISTRITO FEDERAL. AÇÃO JULGADA PROCEDENTE EM RAZÃO DO VÍCIO FORMAL. Tanto o Decreto n.º 10.829/87, quanto a Portaria n.º 314/92, do Instituto Brasileiro do Patrimônio Cultural  IBPC, hoje Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN, conferem ao Governador do Distrito Federal competência privativa para iniciar o processo legislativo, quando se tratar o tema de uso e ocupação do solo em todo o território do Distrito F

In [6]:
#!pip install -U spacy

In [7]:
#!python -m spacy download pt_core_news_sm

In [8]:
import spacy
import pt_core_news_sm

In [9]:
modelo_ner = pt_core_news_sm.load()

In [10]:
doc = modelo_ner(texto)

for entidade in doc.ents:
  print(f'{entidade.text} => {entidade.label_}')

Órgão => LOC
Conselho Especial
Classe => ORG
DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s => LOC
DA CÂMARA => ORG
DISTRITO FEDERAL E OUTRO(S)
Relator => LOC
LEIS DISTRITAIS => MISC
COMPLEMENTAR DISTRITAL => MISC
FORMAL => ORG
LEI ORGÂNICA => MISC
DISTRITO FEDERAL => LOC
ÁREA PÚBLICA => LOC
DO GOVERNADOR => MISC
DISTRITO FEDERAL => LOC
VÍCIO FORMAL => ORG
Decreto n.º 10.829/87 => MISC
Portaria n.º 314/92 => ORG
Instituto Brasileiro do Patrimônio Cultural  IBPC => LOC
Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN => LOC
Governador do Distrito Federal => LOC
Distrito Federal => LOC
ACÓRDÃO => ORG
Acordam os Desembargadores do Conselho Especial do Tribunal de Justiça do Distrito Federal => MISC
Territórios LÉCIO RESENDE  Relator => ORG
JOÃO MAIORISI => PER
Vogal => ORG
VIEIRA  Vogal => LOC
SÉRGIO BITTENCOURT  Vogal => ORG
LECIR MANOEL DA LUZ => ORG
Vogal => LOC
CARMELITA BRASIL => ORG
Vogal => LOC
WALDIR LEÔNCIO => PER
J.J. COSTA CARVALHO => PER
Vogal => LOC
HUMBERTO => LOC
V

In [11]:
import pandas as pd

entidades = []
labels = []

for entidade in doc.ents:
  entidades.append(entidade.text)
  labels.append(entidade.label_)

df = pd.DataFrame({'Entidades': entidades, 'Label': labels})

In [12]:
df.head()

,Entidades,Label
0,Órgão,LOC
1,Conselho Especial\nClasse,ORG
2,DISTRITO FEDERAL E TERRITÓRIOS\nRequerido(s,LOC
3,DA CÂMARA,ORG
4,DISTRITO FEDERAL E OUTRO(S)\nRelator,LOC


Visualizando as entidades no texto

In [13]:
modelo_ner.get_pipe('ner').labels

('LOC', 'MISC', 'ORG', 'PER')

In [14]:
print('LOC', spacy.explain('LOC'))
print('MiSC', spacy.explain('MISC'))
print('ORG', spacy.explain('ORG'))
print('PER', spacy.explain('PER'))

LOC Non-GPE locations, mountain ranges, bodies of water
MiSC Miscellaneous entities, e.g. events, nationalities, products or works of art
ORG Companies, agencies, institutions, etc.
PER Named person or family.


In [15]:
spacy.displacy.render(doc, style='ent', jupyter=True)

In [16]:
dados = []

with zipfile.ZipFile('/content/sample_data/texts.zip') as zip:
  for nome_arquivo in zip.namelist():
    with zip.open(nome_arquivo) as arquivo:
      conteudo = arquivo.read().decode('utf-8')
      palavras = conteudo.split()
      for palavra in palavras:
        dados.append([nome_arquivo, palavra])

tabela_palavras = pd.DataFrame(dados, columns=['arquivo', 'palavra'])


In [17]:
tabela_palavras

,arquivo,palavra
0,AC1TCU.txt,Número
1,AC1TCU.txt,do
2,AC1TCU.txt,Acórdão
3,AC1TCU.txt,ACÓRDÃO
4,AC1TCU.txt,2924/2017
...,...,...
193487,TSTRR16037920105200001.txt,Relator
193488,TSTRR16037920105200001.txt,fls.
193489,TSTRR16037920105200001.txt,PROCESSO
193490,TSTRR16037920105200001.txt,Nº


In [18]:
tabela_palavras.to_csv('palavras.csv', index=False, sep='\t')

In [19]:
tabela_palavras = pd.read_csv('/content/sample_data/palavras_IOB.tsv', sep='\t')

In [20]:
tabela_palavras

,arquivo,palavra,label
0,TCU4687.txt,GRUPO,O
1,TCU4687.txt,I,O
2,TCU4687.txt,CLASSE,O
3,TCU4687.txt,II,O
4,TCU4687.txt,2ª,B-ORGANIZACAO
...,...,...,...
229272,RR2574407120025020372.txt,de,O
229273,RR2574407120025020372.txt,Chaves,O
229274,RR2574407120025020372.txt,Públicas,O
229275,RR2574407120025020372.txt,Brasileira,O


In [21]:
tabela_palavras['label'].unique()

array(['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-JURISPRUDENCIA',
       'I-JURISPRUDENCIA', 'B-PESSOA', 'I-PESSOA', 'B-LEGISLACAO',
       'I-LEGISLACAO', 'B-TEMPO', 'B-LOCAL', 'I-LOCAL', 'I-TEMPO'],
      dtype=object)

Transformando um arquivo para o formato *spacy* **`bold text`**

Formato Spacy:

tupla = ('texto', {'entities': [(10, 17, 'ORGANIZACAO'), (25, 40, 'PESSOA')]})

In [44]:
grupos_arquivo = tabela_palavras.groupby(by='arquivo')

In [45]:
grupos_arquivo.get_group('ADI2TJDFT.txt')

,arquivo,palavra,label
46105,ADI2TJDFT.txt,Órgão,O
46106,ADI2TJDFT.txt,:,O
46107,ADI2TJDFT.txt,Conselho,B-ORGANIZACAO
46108,ADI2TJDFT.txt,Especial,I-ORGANIZACAO
46109,ADI2TJDFT.txt,Classe,O
...,...,...,...
55911,ADI2TJDFT.txt,a,O
55912,ADI2TJDFT.txt,ação,O
55913,ADI2TJDFT.txt,",",O
55914,ADI2TJDFT.txt,maioria,O


In [24]:
tabela_agrupada = grupos_arquivo.get_group('ADI2TJDFT.txt')[['palavra', 'label']].values

conteudo = ''
anotacoes = {'entities': []}
inicio_palavra = 0
fim_palavra = 0

for texto, label in tabela_agrupada:
  texto = str(texto)
  tamanho_texto = len(texto) + 1

  inicio_palavra = fim_palavra
  fim_palavra = inicio_palavra + tamanho_texto

  if label != 'O':
    anotacao = (inicio_palavra, fim_palavra-1, label)
    anotacoes['entities'].append(anotacao)

  conteudo += texto + ' '

tupla = (conteudo, anotacoes)

In [25]:
conteudo

"Órgão : Conselho Especial Classe : ADI Ação Direta de Inconstitucionalidade N . Processo : 2010002019357-4 Requerente ( s ) : PROCURADORA-GERAL DE JUSTIÇA DO DISTRITO FEDERAL E TERRITÓRIOS Requerido ( s ) : PRESIDENTE DA CÂMARA LEGISLATIVA DO DISTRITO FEDERAL E OUTRO ( S ) Relator : Desembargador LÉCIO RESENDE EMENTA AÇÃO DIRETA DE INCONSTITUCIONALIDADE . LEIS DISTRITAIS N.º 747/1994 E 2018/1998 . LEI COMPLEMENTAR DISTRITAL N.º 380/2001 . INCONSTITUCIONALIDADE FORMAL . LEI ORGÂNICA DO DISTRITO FEDERAL . OCUPAÇÃO DE ÁREA PÚBLICA . COMPETÊNCIA PRIVATIVA DO GOVERNADOR DO DISTRITO FEDERAL . AÇÃO JULGADA PROCEDENTE EM RAZÃO DO VÍCIO FORMAL . Tanto o Decreto n.º 10.829/87 , quanto a Portaria n.º 314/92 , do Instituto Brasileiro do Patrimônio Cultural IBPC , hoje Instituto do Patrimônio Histórico e Artístico Nacional IPHAN , conferem ao Governador do Distrito Federal competência privativa para iniciar o processo legislativo , quando se tratar o tema de uso e ocupação do solo em todo o territ

In [26]:
anotacoes

{'entities': [(8, 16, 'B-ORGANIZACAO'),
  (17, 25, 'I-ORGANIZACAO'),
  (91, 106, 'B-JURISPRUDENCIA'),
  (158, 166, 'B-LOCAL'),
  (167, 174, 'I-LOCAL'),
  (221, 227, 'B-ORGANIZACAO'),
  (228, 239, 'I-ORGANIZACAO'),
  (240, 242, 'I-ORGANIZACAO'),
  (243, 251, 'I-ORGANIZACAO'),
  (252, 259, 'I-ORGANIZACAO'),
  (298, 303, 'B-PESSOA'),
  (304, 311, 'I-PESSOA'),
  (358, 362, 'B-LEGISLACAO'),
  (363, 373, 'I-LEGISLACAO'),
  (374, 377, 'I-LEGISLACAO'),
  (378, 386, 'I-LEGISLACAO'),
  (389, 398, 'B-LEGISLACAO'),
  (401, 404, 'B-LEGISLACAO'),
  (405, 417, 'I-LEGISLACAO'),
  (418, 427, 'I-LEGISLACAO'),
  (428, 431, 'I-LEGISLACAO'),
  (432, 440, 'I-LEGISLACAO'),
  (474, 477, 'B-LEGISLACAO'),
  (478, 486, 'I-LEGISLACAO'),
  (487, 489, 'I-LEGISLACAO'),
  (490, 498, 'I-LEGISLACAO'),
  (499, 506, 'I-LEGISLACAO'),
  (575, 583, 'B-LOCAL'),
  (584, 591, 'I-LOCAL'),
  (653, 660, 'B-LEGISLACAO'),
  (661, 664, 'I-LEGISLACAO'),
  (665, 674, 'I-LEGISLACAO'),
  (686, 694, 'B-LEGISLACAO'),
  (695, 698, 'I-LEGIS

In [27]:
conteudo.find('Conselho Especial')

8

In [28]:
conteudo.find('Conselho Especial') + len('Conselho Especial')

25

In [30]:
arquivos = grupos_arquivo.groups.keys()


In [31]:
arquivos

dict_keys(['AC1TCU.txt', 'AC1TJAC.txt', 'AC1TJMG.txt', 'AC2.txt', 'ACORDAOTCU25052016.txt', 'ADI2TJDFT.txt', 'AIAgRAgI6193ARAGUARIMG.txt', 'AIRR15708820115050222.txt', 'AIRR3999520145020086.txt', 'AP00000794920137060006.txt', 'AP00001415620157010201.txt', 'AP00001441420167030203.txt', 'AP771420167080008PA.txt', 'Ag10000170733596001.txt', 'Ag10105170208398001.txt', 'AgAIRR11889820145030011.txt', 'AgCr10582160008758001.txt', 'AgRgSTJ1.txt', 'AgRgSTJ2.txt', 'AgRgTSE1.txt', 'AgRgTSE3.txt', 'CP32320177080008PA.txt', 'DespSEPLAGDF.txt', 'ED1STM.txt', 'ED1TJAC.txt', 'EDAgRgTSE2.txt', 'EDEDARR208420135040232.txt', 'EDRR1TST.txt', 'EEDRR9715120105020002.txt', 'ERR731004520105130003.txt', 'HC110260SP.txt', 'HC151914AgRES.txt', 'HC340624SP.txt', 'HC418951PR.txt', 'HC70000845920187000000.txt', 'Lei11788.txt', 'LoaDF2018.txt', 'Pet128TSE5.txt', 'Port77DF.txt', 'REE5908TSE4.txt', 'REsp1583083RS.txt', 'RR2574407120025020372.txt', 'RR474820145230056.txt', 'RR942006420095040028.txt', 'Rcl3495STJ.txt', 

In [41]:
documentos = []

for arquivo in arquivos:
  documentos = []
  tabela_agrupada = grupos_arquivo.get_group(arquivo)[['palavra', 'label']].values

  conteudo = ''
  anotacoes = {'entities': []}
  inicio_palavra = 0
  fim_palavra = 0

  for texto, label in tabela_agrupada:
    texto = str(texto)
    tamanho_texto = len(texto) + 1

    inicio_palavra = fim_palavra
    fim_palavra = inicio_palavra + tamanho_texto

    if label != 'O':
      anotacao = (inicio_palavra, fim_palavra-1, label)
      anotacoes['entities'].append(anotacao)

    conteudo += texto + ' '

  documento = (conteudo, anotacoes)
  documentos.append(documento)

In [33]:
documentos

[('Brastra.gif ( 4376 bytes ) Presidência da República Casa Civil Subchefia para Assuntos Jurídicos LEI Nº 11.340 , DE 7 DE AGOSTO DE 2006 . Vigência ( Vide ADI nº 4427 ) Cria mecanismos para coibir a violência doméstica e familiar contra a mulher , nos termos do § 8o do art . 226 da Constituição Federal , da Convenção sobre a Eliminação de Todas as Formas de Discriminação contra as Mulheres e da Convenção Interamericana para Prevenir , Punir e Erradicar a Violência contra a Mulher ; dispõe sobre a criação dos Juizados de Violência Doméstica e Familiar contra a Mulher ; altera o Código de Processo Penal , o Código Penal e a Lei de Execução Penal ; e dá outras providências . O PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei : TÍTULO I DISPOSIÇÕES PRELIMINARES Art . 1o Esta Lei cria mecanismos para coibir e prevenir a violência doméstica e familiar contra a mulher , nos termos do § 8o do art . 226 da Constituição Federal , da Convenção sob

In [43]:
len(documentos)

1

In [34]:
import random

random.shuffle(documentos)

In [35]:
documentos

[('Brastra.gif ( 4376 bytes ) Presidência da República Casa Civil Subchefia para Assuntos Jurídicos LEI Nº 11.340 , DE 7 DE AGOSTO DE 2006 . Vigência ( Vide ADI nº 4427 ) Cria mecanismos para coibir a violência doméstica e familiar contra a mulher , nos termos do § 8o do art . 226 da Constituição Federal , da Convenção sobre a Eliminação de Todas as Formas de Discriminação contra as Mulheres e da Convenção Interamericana para Prevenir , Punir e Erradicar a Violência contra a Mulher ; dispõe sobre a criação dos Juizados de Violência Doméstica e Familiar contra a Mulher ; altera o Código de Processo Penal , o Código Penal e a Lei de Execução Penal ; e dá outras providências . O PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei : TÍTULO I DISPOSIÇÕES PRELIMINARES Art . 1o Esta Lei cria mecanismos para coibir e prevenir a violência doméstica e familiar contra a mulher , nos termos do § 8o do art . 226 da Constituição Federal , da Convenção sob

In [37]:
len(documentos)

1